In [1]:
import sys
if "../" not in sys.path:
    sys.path.append("../")

In [41]:
tasks = [(1, 17, 2, 3), [1, 26, 3, 7, 5, 6], [1, 24, 3]]
orders = [
    [(1, 17), (17, 2), (2, 3)],
    [(1, 26), (26, 3), (3, 7), (7, 5)],
    [(1, 24), (24, 3)],
]
import pickle

with open("../datasets/capacities.pickle", "rb") as f:
    capacities = pickle.load(f)

In [39]:
from jobshop_v2.validation import checkCycle
from jobshop_v2.mip_solver import LPJS

In [4]:
import pandas as pd

In [42]:
model = LPJS(tasks, orders, capacities)
model.solve()

Solution:
Objective value =  28.76
Job 0 task 1 starts at 0.00 and ends at 2.16 on machine 25
Job 0 task 17 starts at 2.16 and ends at 6.14 on machine 2
Job 0 task 2 starts at 6.14 and ends at 10.48 on machine 4
Job 0 task 3 starts at 10.48 and ends at 10.76 on machine 5
Job 1 task 1 starts at 2.16 and ends at 4.32 on machine 25
Job 1 task 26 starts at 7.42 and ends at 10.76 on machine 12
Job 1 task 3 starts at 10.76 and ends at 11.04 on machine 5
Job 1 task 7 starts at 11.04 and ends at 17.63 on machine 27
Job 1 task 5 starts at 0.00 and ends at 7.51 on machine 9
Job 1 task 6 starts at -0.00 and ends at 2.85 on machine 3
Job 2 task 1 starts at 4.32 and ends at 6.48 on machine 25
Job 2 task 24 starts at 6.48 and ends at 15.64 on machine 18
Job 2 task 3 starts at 15.64 and ends at 15.92 on machine 5


## Mở file Excel sau đây để xem chi tiết các công việc và số thứ tự của chúng

In [305]:
import ipywidgets as widgets
from IPython.display import display
import os
def open_data_file(_):
    file_path = r"..\datasets\Data.xlsx"
    if os.path.exists(file_path):
        os.startfile(file_path)
    else:
        print("File not found.")

w = widgets.Button(description='Open data file')
w.on_click(open_data_file)
display(w)


Button(description='Open data file', style=ButtonStyle())

In [5]:
%%html
<style>
.mytext > .widget-label {
    color: #066889;
    font-size: 20px;
}
</style>

In [49]:
import ipywidgets as widgets
from ipywidgets import Layout, Button, Box, VBox, HBox, Textarea, Label, Text

# Step 2
num_jobs = widgets.IntSlider(
    value=1,
    min=1,
    max=100,
    description="Chọn số lượng công việc:",
    style={"description_width": "initial", "handle_color": "lightblue"},
    layout=Layout(width="50%"),
)
num_jobs.add_class("mytext")
output = widgets.Output()
tasks_input = []
orders_input = []


# Step 5
def on_button_click(b):
    output.clear_output()
    global tasks_input, orders_input
    tasks_input = [
        Text(
            value="",
            description="Các tác vụ:",
            placeholder="e.g. 1, 17, 2, 3",
            style={"description_width": "initial", "font_size": "20px"},
            layout=Layout(width="45%"),
        )
        for _ in range(num_jobs.value)
    ]
    orders_input = [
        Text(
            value="",
            description="Thứ tự thực hiện:",
            placeholder="e.g. (1, 17), (17, 2), (2, 3)",
            style={"description_width": "initial", "font_size": "20px"},
            layout=Layout(width="45%"),
        )
        for _ in range(num_jobs.value)
    ]
    # Step 6
    num = num_jobs.value
    for i in range(num):
        task = tasks_input[i]
        task.add_class("mytext")
        order = orders_input[i]
        order.add_class("mytext")
        tasks_orders = VBox(
            [
                Label(
                    "Công việc %d:" % (i + 1),
                    style={"font_size": "20px", "text_color": "#0fa312"},
                ),
                HBox(
                    [task, order],
                    layout=widgets.Layout(
                        display="flex",
                        justify_content="space-around",
                    ),
                ),
            ]
        )
        # Step 9
        with output:
            display(tasks_orders)


# Step 8
button = widgets.Button(
    description="Nhấn để nhập thông tin về các công việc",
    style={"font_weight": "bold", "font_size": "15px", "button_color": "pink"},
    layout=widgets.Layout(width="25%", height="50px"),
)
button.add_class("mytext")
button.on_click(on_button_click)
solve_button = Button(
    description="Solve",
    button_style="success",
    tooltip="Click me",
    icon="check",
    style={"font_weight": "bold", "font_size": "15px"},
)
display(num_jobs, button, output, solve_button)

tasks = []
orders = []
model = None
def bind_solve_button(b):
    #clear all errors output
    global tasks, orders, tasks_input, orders_input
    tasks = []
    orders = []
    for task in tasks_input:
        if task.value == "":
            raise ValueError("Các tác vụ không được để trống")
        try:
            tasks.append(eval(task.value))
        except:
            tasks.append([eval(task.value)])
    for order in orders_input:
        if order.value == "":
            continue
        l = eval(order.value)
        if type(l[0]) == int:
            orders.append([l])
        else:
            orders.append(l)
    assert all([not checkCycle(order) for order in orders]), "Tồn tại chu trình trong các thứ tự thực hiện"
    df = pd.read_excel("../datasets/Data.xlsx", sheet_name="Tasks")
    all_tasks = set(df["Task ID"])
    for task in tasks:
        assert all([t in all_tasks for t in task]), "Có tác vụ không tồn tại trong bảng Tasks"
    for i, order in enumerate(orders):
        tasks_in_job = set.union(*[set(i) for i in order])
        assert all([t in tasks[i] for t in tasks_in_job]), "Có tác vụ không tồn tại trong danh sách tác vụ của công việc"
    global model
    
    model = LPJS(tasks, orders, capacities)
    
    model.solve()


solve_button.on_click(bind_solve_button)

IntSlider(value=1, description='Chọn số lượng công việc:', layout=Layout(width='50%'), min=1, style=SliderStyl…

Button(description='Nhấn để nhập thông tin về các công việc', layout=Layout(height='50px', width='25%'), style…

Output()

Button(button_style='success', description='Solve', icon='check', style=ButtonStyle(font_size='15px', font_wei…

Solution:
Objective value =  3.98
Job 0 task 1 starts at 0.00 and ends at 2.16 on machine 25
Job 0 task 17 starts at 0.00 and ends at 3.98 on machine 2


In [50]:
model.assignment_task

defaultdict(list,
            {(0, 1): [(job_0_task_1_machine_25, 25, 1.76),
              (job_0_task_1_machine_26, 26, 5.15)],
             (0, 17): [(job_0_task_17_machine_1, 1, 7.35),
              (job_0_task_17_machine_2, 2, 2.22),
              (job_0_task_17_machine_6, 6, 4.35),
              (job_0_task_17_machine_11, 11, 3.08),
              (job_0_task_17_machine_13, 13, 5.27)]})

In [ ]:
capa

In [ ]:
# tasks = [(1, 17, 2, 3), [1, 26, 3, 7, 5, 6], [1, 24, 3]]
# orders = [
#     [(1, 17), (17, 2), (2, 3)],
#     [(1, 26), (26, 3), (3, 7), (7, 5)],
#     [(1, 24), (24, 3)],
# ]